In [1]:
import numpy as np
import pandas as pd
import zipfile
import os

import matplotlib.pyplot as plt
%matplotlib inline

### Data generation

In [2]:
!mkdir ~/.kaggle
!cp "/content/kaggle.json" ~/.kaggle/kaggle.json 
os.environ["KAGGLE_CONFIG_"] ='/content/kaggle.json'
!kaggle competitions download -c kernel-methods-ammi-2022

  0% 0.00/464k [00:00<?, ?B/s]
100% 464k/464k [00:00<00:00, 75.1MB/s]


In [3]:
!unzip "/content/kernel-methods-ammi-2022.zip" -d data/

Archive:  /content/kernel-methods-ammi-2022.zip
  inflating: data/Xte.csv            
  inflating: data/Xte_vectors.csv    
  inflating: data/Xtr.csv            
  inflating: data/Xtr_vectors.csv    
  inflating: data/Ytr.csv            


In [4]:
x=pd.read_csv('/content/data/Xtr_vectors.csv')
x.head()

,Id,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,1,0.1406,0.0625,0.0781,0.1094,0.0625,0.0312,0.0312,0.1406,0.0781,...,0.0625,0.1719,0.0781,0.0312,0.0625,0.1719,0.0781,0.1094,0.1406,0.1250
1,2,0.1250,0.1250,0.0781,0.0938,0.0625,0.0469,0.0625,0.1094,0.1406,...,0.0781,0.1406,0.1094,0.0625,0.0312,0.1406,0.0625,0.1406,0.1250,0.1406
2,3,0.1406,0.1719,0.0156,0.0781,0.1406,0.1094,0.0938,0.0469,0.0312,...,0.1406,0.0938,0.0938,0.0156,0.0625,0.0781,0.1562,0.0469,0.0781,0.1094
3,4,0.1406,0.0938,0.0781,0.1250,0.1250,0.0469,0.0312,0.0781,0.0938,...,0.0312,0.2031,0.1094,0.0469,0.0469,0.1875,0.1094,0.1562,0.1406,0.1875
4,5,0.2188,0.1094,0.1094,0.2031,0.1250,0.0625,0.0156,0.1094,0.1406,...,0.0312,0.0938,0.0781,0.0156,0.0312,0.1250,0.1719,0.0781,0.0625,0.1719


In [5]:
sub=pd.read_csv("/content/data/Xte_vectors.csv")
sub.tail()

,Id,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
995,996,0.1562,0.1406,0.0625,0.0781,0.0625,0.0156,0.1094,0.0938,0.1094,...,0.0625,0.0312,0.0781,0.0625,0.1562,0.0625,0.0625,0.1094,0.0625,0.0938
996,997,0.0781,0.1094,0.1094,0.1562,0.1562,0.0312,0.0781,0.0312,0.0781,...,0.0938,0.0625,0.0625,0.0312,0.0938,0.1562,0.1875,0.0781,0.1406,0.0469
997,998,0.0781,0.0625,0.0781,0.0469,0.0312,0.0938,0.0312,0.0938,0.0312,...,0.0469,0.0781,0.0469,0.0781,0.1250,0.0469,0.0469,0.0781,0.1250,0.0312
998,999,0.0781,0.0781,0.0938,0.0312,0.0469,0.0938,0.0469,0.0625,0.0938,...,0.1406,0.0625,0.0312,0.0781,0.0312,0.0469,0.0469,0.0156,0.0312,0.0469
999,1000,0.0938,0.1094,0.0312,0.0469,0.1094,0.0938,0.1406,0.0938,0.0312,...,0.0625,0.0938,0.0469,0.0625,0.0469,0.0625,0.0781,0.0000,0.0938,0.0469


In [6]:
y=pd.read_csv("/content/data/Ytr.csv")
y.describe()

,Id,Covid
count,2000.000000,2000.000000
mean,1000.500000,0.500000
std,577.494589,0.500125
min,1.000000,0.000000
25%,500.750000,0.000000
50%,1000.500000,0.500000
75%,1500.250000,1.000000
max,2000.000000,1.000000


In [7]:
y['Covid'].replace(0,-1,inplace=True)
y.head()

,Id,Covid
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [8]:
x.drop(['Id'],axis=1,inplace=True)
y.drop(['Id'],axis=1,inplace=True)
y.head()

,Covid
0,1
1,1
2,1
3,1
4,1


In [9]:
dataset=pd.concat([x,y["Covid"]],axis=1)
dataset.head()

,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,Covid
0,0.1406,0.0625,0.0781,0.1094,0.0625,0.0312,0.0312,0.1406,0.0781,0.0312,...,0.1719,0.0781,0.0312,0.0625,0.1719,0.0781,0.1094,0.1406,0.1250,1
1,0.1250,0.1250,0.0781,0.0938,0.0625,0.0469,0.0625,0.1094,0.1406,0.0156,...,0.1406,0.1094,0.0625,0.0312,0.1406,0.0625,0.1406,0.1250,0.1406,1
2,0.1406,0.1719,0.0156,0.0781,0.1406,0.1094,0.0938,0.0469,0.0312,0.0000,...,0.0938,0.0938,0.0156,0.0625,0.0781,0.1562,0.0469,0.0781,0.1094,1
3,0.1406,0.0938,0.0781,0.1250,0.1250,0.0469,0.0312,0.0781,0.0938,0.0312,...,0.2031,0.1094,0.0469,0.0469,0.1875,0.1094,0.1562,0.1406,0.1875,1
4,0.2188,0.1094,0.1094,0.2031,0.1250,0.0625,0.0156,0.1094,0.1406,0.0469,...,0.0938,0.0781,0.0156,0.0312,0.1250,0.1719,0.0781,0.0625,0.1719,1


In [10]:
x.shape,y.shape

((2000, 64), (2000, 1))

In [11]:
def error(ypred, ytrue):
    e = (ypred != ytrue).mean()
    return e

def add_column_ones(X):
    n = X.shape[0]
    return np.hstack([X, np.ones((n, 1))])


In [12]:
def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the RBF kernel with parameter sigma
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    sigma: float
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

In [13]:
def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix

    '''
    linear=np.dot(X1,X2.T)

    return linear

def polynomial_kernel(X1, X2, degree=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the polynomial kernel of degree `degree`
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    polynomial=(linear_kernel(X1, X2)+1)**degree
    return polynomial

def sigmoid(self,x):
    sig = 1/(1+np.exp(-x))
    return sig

In [14]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    fit_K
    predict_K
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel#,
        # 'mismatch': mismatch_kernel,
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        self.fit_intercept_ = False
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', 1.)
        if self.kernel_name == 'polynomial':
            params['degree'] = kwargs.get('degree', 2)
        return params

    def fit_K(self, K, y, **kwargs):
        pass
        
    def decision_function_K(self, K):
        pass
    
    def fit(self, X, y, fit_intercept=False, **kwargs):

        if fit_intercept:
            X = add_column_ones(X)
            self.fit_intercept_ = True
        self.X_train = X
        self.y_train = y

        K = self.kernel_function_(self.X_train, self.X_train, **self.kernel_parameters)

        return self.fit_K(K, y, **kwargs)
    
    def decision_function(self, X):

        if self.fit_intercept_:
            X = add_column_ones(X)

        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)

        return self.decision_function_K(K_x)

    def predict(self, X):
        pass
    
    def predict_K(self, K):
        pass

In [15]:
import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

In [16]:
def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-3
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :].astype(float)
    b = np.array([0.])
    return P, q, G, h, A, b

# K = linear_kernel(x_train, x_train)
# alphas = solve_qp(*svm_dual_soft_to_qp_kernel(K, y_train["Covid"], C=1.))

In [17]:
class KernelSVM(KernelMethodBase):
    '''
    Kernel SVM Classification
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        super().__init__(**kwargs)
    
    def fit_K(self, K, y, tol=1e-3):
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.C - self.alpha > tol))
        self.bias = y[sv] - K[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        self.beta=self.alpha*y
        
        return self
        
    def decision_function_K(self, K_x):
        return K_x@(self.alpha*self.beta)+self.bias

    def predict(self, X):
        return np.sign(self.decision_function(X))

In [18]:
def accuracy(y_test,y_pred):
  count=0
  for i,(value) in enumerate(zip(y_test,y_pred)):
    if value[0]==value[1]:
      count+=1
  accuracy=(count/y_pred.shape[0])*100
  return accuracy


In [19]:

fold_score=[]
sub_fold_pred=[]


X=dataset
n_folds=5
np.random.seed(2022)
n=X.shape[0]

X_train=X.drop(["Covid"
],axis=1)

In [20]:
Y_train=y
fold_size=len(X)//n_folds
print(len(X),fold_size)

2000 400


In [21]:
kernel = 'rbf'
sigma = 0.1
degree = 2
CL = np.arange(1.0,3.5,0.5)
tol = 1e-3
cvsL = np.zeros(len(CL))

In [26]:
fold_score=[]
sub_fold_pred=[]
for folds in range(n_folds):
  indices=np.arange(folds*fold_size,(1+folds)*fold_size, dtype=int)
  x_val=X_train.iloc[indices]
  y_val=Y_train.iloc[indices]
  x_train=X_train[~X_train.apply(tuple,1).isin(x_val.apply(tuple,1))]
  y_train=Y_train.iloc[x_train.index]
  print(f"fold:{folds}")
  for i,c in enumerate(CL):
    model = KernelSVM(C=c, kernel=kernel, sigma=sigma, degree=degree)
    print(f"C:{c}")
    y_pred = model.fit(x_train.to_numpy(), y_train["Covid"].to_numpy(), tol=tol).predict(x_val.to_numpy())
    acc=accuracy(y_val['Covid'].to_numpy(),y_pred)
    print("accuracy score: ", acc)
    sub_pred=model.predict(sub.iloc[:,1:].to_numpy())
    sub_fold_pred.append(sub_pred)
    cvsL[i] = acc
  best_index = np.argmax(cvsL)

fold:0
C:1.0
accuracy score:  91.0
C:1.5
accuracy score:  90.75
C:2.0
accuracy score:  90.75
C:2.5
accuracy score:  90.75
C:3.0
accuracy score:  90.75
fold:1
C:1.0
accuracy score:  87.0
C:1.5
accuracy score:  87.5
C:2.0
accuracy score:  87.5
C:2.5
accuracy score:  87.5
C:3.0
accuracy score:  87.5
fold:2
C:1.0
accuracy score:  88.0
C:1.5
accuracy score:  87.75
C:2.0
accuracy score:  87.75
C:2.5
accuracy score:  87.75
C:3.0
accuracy score:  87.75
fold:3
C:1.0
accuracy score:  86.75
C:1.5
accuracy score:  87.0
C:2.0
accuracy score:  87.25
C:2.5
accuracy score:  87.25
C:3.0
accuracy score:  87.25
fold:4
C:1.0
accuracy score:  86.0
C:1.5
accuracy score:  87.25
C:2.0
accuracy score:  87.25
C:2.5
accuracy score:  87.25
C:3.0
accuracy score:  87.25


In [22]:
fold_score=[]
sub_fold_pred=[]
for folds in range(n_folds):
  indices=np.arange(folds*fold_size,(1+folds)*fold_size, dtype=int)
  x_val=X_train.iloc[indices]
  y_val=Y_train.iloc[indices]
  x_train=X_train[~X_train.apply(tuple,1).isin(x_val.apply(tuple,1))]
  y_train=Y_train.iloc[x_train.index]
  print(f"fold:{folds}")

  model = KernelSVM(C=1, kernel=kernel, sigma=sigma, degree=degree)
  y_pred = model.fit(x_train.to_numpy(), y_train["Covid"].to_numpy(), tol=tol).predict(x_val.to_numpy())
  acc=accuracy(y_val['Covid'].to_numpy(),y_pred)
  print("accuracy score: ", acc)
  fold_score.append(acc)
  sub_pred=model.predict(sub.iloc[:,1:].to_numpy())
  sub_fold_pred.append(sub_pred)
np.mean(fold_score)

fold:0
accuracy score:  91.0
fold:1
accuracy score:  87.0
fold:2
accuracy score:  88.0
fold:3
accuracy score:  86.75
fold:4
accuracy score:  86.0


87.75

In [23]:
sub_pred=pd.DataFrame(np.sign(np.mean(sub_fold_pred, 0)).astype(int),columns=['Covid'])
ss_pred=pd.concat([sub['Id'],sub_pred],axis=1)
ss_pred.tail()

,Id,Covid
995,996,-1
996,997,-1
997,998,-1
998,999,-1
999,1000,-1


In [24]:
ss_pred.describe()

,Id,Covid
count,1000.000000,1000.000000
mean,500.500000,-0.222000
std,288.819436,0.975535
min,1.000000,-1.000000
25%,250.750000,-1.000000
50%,500.500000,-1.000000
75%,750.250000,1.000000
max,1000.000000,1.000000


In [25]:
ss_pred['Covid'].replace(-1,0,inplace=True)


In [26]:
ss_pred["Covid"].value_counts()

0    611
1    389
Name: Covid, dtype: int64

In [27]:
sample_sub=ss_pred.to_csv('10_sub.csv',index=False)
pd.read_csv('/content/10_sub.csv')

,Id,Covid
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1
...,...,...
995,996,0
996,997,0
997,998,0
998,999,0


In [ ]:
# !kaggle competitions submit -c kernel-methods-ammi-2022 -f 10_sub.csv -m "10th submission"

100% 5.76k/5.76k [00:02<00:00, 2.46kB/s]
Successfully submitted to Kernel Methods AMMI 2022